# **Machine Learning Class Task Two**

**GROUP MEMBERS**

---



SUSAN GICHEHA - 119988

KATHERINE MUNG'AYI - 121195

MARIA GITAU - 102851

TREVOR OMONDI - 121288


In [128]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from numpy.linalg import svd
from sklearn import linear_model,metrics
from sklearn.linear_model import Lasso,LassoCV
from sklearn.model_selection import train_test_split,KFold,cross_val_score,GridSearchCV,RandomizedSearchCV
from sklearn.decomposition import TruncatedSVD, PCA

## **Loading the dataset**


In [79]:
data=pd.read_csv("modified_data.csv")

In [80]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [81]:
data.shape

(1460, 77)

In [82]:
data.drop('Id',inplace=True,axis=1)

In [83]:
original_features=list(data.columns)
len(original_features)

76

In [84]:
features_missing_values=list(data.columns[data.isna().any()])
len(features_missing_values)

15

In [85]:
len(data)

1460

## **Fill in Missing Values**

In [86]:
data.isna().sum()

MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 76, dtype: int64

In [87]:
bfill,mean_fill=[],[]

In [88]:
for feature in features_missing_values:
  print(feature,' ',data[feature].isna().sum()/len(data)*100,' ',data[feature].dtype)
  if data[feature].dtype=='float64':
    mean_fill.append(feature)
  else:
    bfill.append(feature)

LotFrontage   17.73972602739726   float64
MasVnrType   0.547945205479452   object
MasVnrArea   0.547945205479452   float64
BsmtQual   2.5342465753424657   object
BsmtCond   2.5342465753424657   object
BsmtExposure   2.6027397260273974   object
BsmtFinType1   2.5342465753424657   object
BsmtFinType2   2.6027397260273974   object
Electrical   0.0684931506849315   object
FireplaceQu   47.26027397260274   object
GarageType   5.5479452054794525   object
GarageYrBlt   5.5479452054794525   float64
GarageFinish   5.5479452054794525   object
GarageQual   5.5479452054794525   object
GarageCond   5.5479452054794525   object


In [89]:
mean_fill

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [90]:
for feature in mean_fill:
  data[feature].fillna(data[feature].mean(),inplace=True)

In [91]:
for feature in bfill:
  data[feature].fillna(method='bfill',inplace=True)

In [92]:
data.columns[data.isna().any()]

Index(['FireplaceQu'], dtype='object')

## **One Hot Encoding**

In [93]:
encoding = data.select_dtypes(include='O').keys()
encoding

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition'],
      dtype='object')

In [94]:
categorical_features=list(data.select_dtypes(include=['object']).columns)
categorical_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [95]:
numerical=list(set(original_features)-set(categorical_features))
numerical

['MiscVal',
 '2ndFlrSF',
 'BsmtFullBath',
 'LowQualFinSF',
 'HalfBath',
 'EnclosedPorch',
 'BedroomAbvGr',
 'LotFrontage',
 'SalePrice',
 'OverallQual',
 'GrLivArea',
 'MoSold',
 'TotRmsAbvGrd',
 '3SsnPorch',
 'KitchenAbvGr',
 'TotalBsmtSF',
 'BsmtFinSF1',
 'Fireplaces',
 'BsmtFinSF2',
 'GarageYrBlt',
 'LotArea',
 'YrSold',
 'FullBath',
 'GarageArea',
 'OpenPorchSF',
 'GarageCars',
 'ScreenPorch',
 'YearRemodAdd',
 'PoolArea',
 '1stFlrSF',
 'OverallCond',
 'YearBuilt',
 'MSSubClass',
 'BsmtUnfSF',
 'WoodDeckSF',
 'BsmtHalfBath',
 'MasVnrArea']

In [96]:
nominal=['MSZoning','LandContour','Neighborhood']
ordinal=list(set(categorical_features)-set(nominal))

In [97]:
target=['SalePrice']

In [98]:
df_nominal=pd.get_dummies(data[nominal])

In [99]:
for feature in ordinal:
  data[feature]=data[feature].astype('category').cat.codes

df_ordinal=data[ordinal]

In [100]:
data[numerical]

,MiscVal,2ndFlrSF,BsmtFullBath,LowQualFinSF,HalfBath,EnclosedPorch,BedroomAbvGr,LotFrontage,SalePrice,OverallQual,...,YearRemodAdd,PoolArea,1stFlrSF,OverallCond,YearBuilt,MSSubClass,BsmtUnfSF,WoodDeckSF,BsmtHalfBath,MasVnrArea
0,0,854,1,0,1,0,3,65.0,208500,7,...,2003,0,856,5,2003,60,150,0,0,196.0
1,0,0,0,0,0,0,3,80.0,181500,6,...,1976,0,1262,8,1976,20,284,298,1,0.0
2,0,866,1,0,1,0,3,68.0,223500,7,...,2002,0,920,5,2001,60,434,0,0,162.0
3,0,756,1,0,0,272,3,60.0,140000,7,...,1970,0,961,5,1915,70,540,0,0,0.0
4,0,1053,1,0,1,0,4,84.0,250000,8,...,2000,0,1145,5,2000,60,490,192,0,350.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,694,0,0,1,0,3,62.0,175000,6,...,2000,0,953,5,1999,60,953,0,0,0.0
1456,0,0,1,0,0,0,3,85.0,210000,6,...,1988,0,2073,6,1978,20,589,349,0,119.0
1457,2500,1152,0,0,0,0,4,66.0,266500,7,...,2006,0,1188,9,1941,70,877,0,0,0.0
1458,0,0,1,0,0,112,2,68.0,142125,5,...,1996,0,1078,6,1950,20,0,366,0,0.0


In [101]:
new_data=pd.concat([df_nominal,df_ordinal,data[numerical]],axis=1)

In [102]:
new_data.shape

(1460, 107)

## **Standardize Data**

In [130]:
new_data.drop('SalePrice',axis=1,inplace=True)

KeyError: ignored

In [131]:
X=new_data.to_numpy()

In [132]:
X

array([[  0.,   0.,   0., ...,   0.,   0., 196.],
       [  0.,   0.,   0., ..., 298.,   1.,   0.],
       [  0.,   0.,   0., ...,   0.,   0., 162.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 366.,   0.,   0.],
       [  0.,   0.,   0., ..., 736.,   0.,   0.]])

In [133]:
X=StandardScaler().fit_transform(X)

In [134]:
y=data[target]

In [135]:
y=y.to_numpy()

In [136]:
y.shape

(1460, 1)

In [137]:
y

array([[208500],
       [181500],
       [223500],
       ...,
       [266500],
       [142125],
       [147500]])

In [138]:
y=StandardScaler().fit_transform(y)

In [139]:
y

array([[ 0.34727322],
       [ 0.00728832],
       [ 0.53615372],
       ...,
       [ 1.07761115],
       [-0.48852299],
       [-0.42084081]])

In [140]:
X.shape

(1460, 106)

**Feature extraction using PCA**

In [141]:
pca = PCA(n_components=2)

In [142]:
p_components=pca.fit_transform(X)

In [143]:
p_components

array([[ 2.44422918, -1.28435444],
       [ 0.31735473,  2.63474496],
       [ 2.8581278 , -0.81652114],
       ...,
       [ 1.26129428, -0.85428088],
       [-2.82789225,  3.27222263],
       [-0.75426088,  2.7670699 ]])

In [144]:
pca.explained_variance_ratio_

array([0.10560106, 0.04471871])

**Train a regression model using PCA with 2 components**



In [145]:
reg=LassoCV()

In [146]:
reg.fit(p_components,y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV()

In [147]:
reg.coef_

array([0.25387431, 0.01904169])

In [148]:
reg.intercept_

1.5341562856749719e-16

**Data splitting**

In [149]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [150]:
y_train

array([[ 1.52462833],
       [-0.69786554],
       [ 0.65791868],
       ...,
       [-0.3767687 ],
       [ 0.99576293],
       [-0.60972131]])

**Picking a model**

In [151]:
result = {}
models = {'LinearRegression':linear_model.LinearRegression(),
          'Lasso':linear_model.Lasso(),
          'Ridge':linear_model.Ridge(),
          'ElasticNet':linear_model.ElasticNet()}

for i in models:
  models[i].fit(X_train,y_train)
  accuracy = models[i].score(X_test,y_test)
  result[i]=[accuracy]
pd.DataFrame(result)

,LinearRegression,Lasso,Ridge,ElasticNet
0,0.742781,-0.000834,0.748295,0.328189


**Training the model**


In [153]:
kfold = KFold(n_splits=4)
for i in models:
  regressor = models[i]
  c_validated = cross_val_score(regressor,X_train,y_train,cv=kfold)  
  accuracy = c_validated.mean()
  result[i]=[accuracy]
pd.DataFrame(result)

,LinearRegression,Lasso,Ridge,ElasticNet
0,-2.623918e+22,-0.005458,0.851916,0.332695


In [158]:
for i in models:
  regressor = models[i]
  if i == 'Lasso':
    lasso_params = {'alpha':[0.02, 0.024, 0.03, 0.035],'max_iter':[500,600,1000],'tol':[0.01, 0.001,0.0001]}
    grid = GridSearchCV(regressor,param_grid=lasso_params,cv=2,n_jobs=-1)
    results = grid.fit(X_train,y_train)
    print("Best Parameters are ", results.best_params_)
    #accuracy = results.score(X_test,y_test)
    accuracy=results.best_score_
  else:    
    c_validated = cross_val_score(regressor,X_train,y_train,cv=kfold)  
    accuracy = c_validated.mean()
  result[i]=[accuracy]
pd.DataFrame(result)

Best Parameters are  {'alpha': 0.02, 'max_iter': 500, 'tol': 0.01}


,LinearRegression,Lasso,Ridge,ElasticNet
0,-2.623918e+22,0.859245,0.851916,0.332695


In [159]:
for i in models:
  regressor = models[i]
  if i == 'Lasso':
    random = RandomizedSearchCV(regressor,param_distributions=lasso_params,cv=3,n_jobs=-1)
    results = random.fit(X_train,y_train)
    print(results.best_score_)
    print("Best Paramaters  are ", results.best_params_)
    accuracy=results.best_score_
  else:    
    c_validated = cross_val_score(regressor,X_train,y_train,cv=kfold)  
    accuracy = c_validated.mean()
  result[i]=[accuracy]
pd.DataFrame(result)

0.8592312624680805
Best Paramaters  are  {'tol': 0.01, 'max_iter': 1000, 'alpha': 0.02}


,LinearRegression,Lasso,Ridge,ElasticNet
0,-2.623918e+22,0.859231,0.851916,0.332695
